In [3]:
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_groq import ChatGroq


c:\Users\Kashif\AppData\Local\Programs\Python\Python310\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
# Initialize Pinecone
def initialize_pinecone(api_key, environment):
    return Pinecone(api_key=api_key, environment=environment)


In [5]:
# Query Pinecone for relevant contexts
def query_pinecone(index, namespace, query_embedding, top_k=6):
    results = index.query(
        namespace=namespace,
        vector=query_embedding[0].values,
        top_k=top_k,
        include_metadata=True
    )
    return results


In [6]:
# Generate embeddings using Pinecone inference
def generate_embeddings(pc, model, inputs, input_type):
    return pc.inference.embed(
        model=model,
        inputs=inputs,
        parameters={"input_type": input_type, "truncate": "END"}
    )

In [7]:
# Use ChatGroq to generate a quiz
def generate_quiz_with_groq(llm, retrieved_context):
    groq_prompt = f"""
    Based on the following context, create a quiz with 10 multiple-choice questions:
    {retrieved_context}
    """
    response = llm.invoke(groq_prompt)
    return response.content


In [ ]:
# Main workflow
def main():
    # Initialize Pinecone
    pc = initialize_pinecone(
        api_key="pcsk_3CYnJi_TZbGr8CeCcVxAsz4Li7J5n5hNBRqM7PA7k6xGKx7ftNXUYMYUJLJcb3PZrTneH4",
        environment="us-west1-gcp"
    )


    # Use ChatGroq to generate the quiz
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.1-70b-versatile"
    )
    
    query = "Explain regression."


    # Generate embedding for the query
    #query_embedding = generate_embeddings(pc, model=model_name, inputs=[query], input_type="query")

    # Query the Pinecone index
    #results = query_pinecone(index, namespace, query_embedding)
    # Retrieve the context
    #retrieved_context = "\n".join([match["metadata"]["text"] for match in results["matches"]])
    #print("Retrieved Context:", retrieved_context)

    
    #quiz = generate_quiz_with_groq(llm, retrieved_context)
    #print("Generated Quiz:", quiz)
    #def delete_all_indexes(pc):
    all_indexes = pc.list_indexes().names()
    print(all_indexes)

if __name__ == "__main__":
    main()


['mcq-index']
